In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [2]:
from pyspark.ml.clustering import KMeans

In [3]:
dataset = spark.read.format('libsvm').load('/FileStore/tables/sample_kmeans_data.txt')

In [4]:
dataset.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0| (3,[],[])|
 1.0|(3,[0,1,2],[0.1,0...|
 2.0|(3,[0,1,2],[0.2,0...|
 3.0|(3,[0,1,2],[9.0,9...|
 4.0|(3,[0,1,2],[9.1,9...|
 5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+

In [5]:
final_data = dataset.select('features')

In [6]:
final_data.show()

+--------------------+
 features|
+--------------------+
 (3,[],[])|
(3,[0,1,2],[0.1,0...|
(3,[0,1,2],[0.2,0...|
(3,[0,1,2],[9.0,9...|
(3,[0,1,2],[9.1,9...|
(3,[0,1,2],[9.2,9...|
+--------------------+

In [7]:
kmeans = KMeans().setK(2).setSeed(1) # seed will have same random centroid

In [8]:
model = kmeans.fit(final_data)

In [9]:
wssse = model.computeCost(final_data)

In [10]:
print(wssse)

0.11999999999994547

In [11]:
final_data.show()

+--------------------+
 features|
+--------------------+
 (3,[],[])|
(3,[0,1,2],[0.1,0...|
(3,[0,1,2],[0.2,0...|
(3,[0,1,2],[9.0,9...|
(3,[0,1,2],[9.1,9...|
(3,[0,1,2],[9.2,9...|
+--------------------+

In [12]:
centers = model.clusterCenters()

In [13]:
centers

Out[ 15 ]: [array([ 0.1, 0.1, 0.1]), array([ 9.1, 9.1, 9.1])]

In [14]:
final_data.show()

+--------------------+
 features|
+--------------------+
 (3,[],[])|
(3,[0,1,2],[0.1,0...|
(3,[0,1,2],[0.2,0...|
(3,[0,1,2],[9.0,9...|
(3,[0,1,2],[9.1,9...|
(3,[0,1,2],[9.2,9...|
+--------------------+

In [15]:
results = model.transform(final_data)

In [16]:
results.show()

+--------------------+----------+
 features|prediction|
+--------------------+----------+
 (3,[],[])| 0|
(3,[0,1,2],[0.1,0...| 0|
(3,[0,1,2],[0.2,0...| 0|
(3,[0,1,2],[9.0,9...| 1|
(3,[0,1,2],[9.1,9...| 1|
(3,[0,1,2],[9.2,9...| 1|
+--------------------+----------+

In [17]:
kmeans = KMeans().setK(3).setSeed(1)

In [18]:
model = kmeans.fit(final_data)

In [19]:
results = model.transform(final_data)

In [20]:
results.show()

+--------------------+----------+
 features|prediction|
+--------------------+----------+
 (3,[],[])| 1|
(3,[0,1,2],[0.1,0...| 1|
(3,[0,1,2],[0.2,0...| 2|
(3,[0,1,2],[9.0,9...| 0|
(3,[0,1,2],[9.1,9...| 0|
(3,[0,1,2],[9.2,9...| 0|
+--------------------+----------+

In [21]:
centers = model.clusterCenters()

In [22]:
centers

Out[ 25 ]: 
[array([ 9.1, 9.1, 9.1]),
 array([ 0.05, 0.05, 0.05]),
 array([ 0.2, 0.2, 0.2])]

In [23]:
#Real life example

spark = SparkSession.builder.appName('cluster').getOrCreate()


In [24]:
dataset = spark.read.csv('/FileStore/tables/seeds_dataset.csv',
                       inferSchema=True,
                       header=True)

In [25]:
dataset.printSchema()

root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)

In [26]:
dataset.head(1)

Out[ 30 ]: [Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)]

In [27]:
from pyspark.ml.clustering import KMeans

In [28]:
from pyspark.ml.feature import VectorAssembler

In [29]:
assembler = VectorAssembler(inputCols=dataset.columns, outputCol='features')

In [30]:
final_data = assembler.transform(dataset)

In [31]:
final_data.printSchema()

root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)
-- features: vector (nullable = true)

In [32]:
from pyspark.ml.feature import StandardScaler

In [33]:
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

In [34]:
scaler_model = scaler.fit(final_data)

In [35]:
final_data = scaler_model.transform(final_data)

In [36]:
final_data.head(1)

Out[ 44 ]: [Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [37]:
kmeans = KMeans(featuresCol='scaledFeatures', k = 3)

In [38]:
model = kmeans.fit(final_data)

In [39]:
print('WSSSE')
print(model.computeCost(final_data))

WSSSE
428.60820118716356

In [40]:
centers = model.clusterCenters()

In [41]:
centers

Out[ 49 ]: 
[array([ 4.96198582, 10.97871333, 37.30930808, 12.44647267,
 8.62880781, 1.80061978, 10.41913733]),
 array([ 6.35645488, 12.40730852, 37.41990178, 13.93860446,
 9.7892399 , 2.41585013, 12.29286107]),
 array([ 4.07497225, 10.14410142, 35.89816849, 11.80812742,
 7.54416916, 3.15410901, 10.38031464])]

In [42]:
results = model.transform(final_data)

In [43]:
results.show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+----------+
 area|perimeter|compactness| length_of_kernel| width_of_kernel|asymmetry_coefficient| length_of_groove| features| scaledFeatures|prediction|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+----------+
15.26| 14.84| 0.871| 5.763| 3.312| 2.221| 5.22|[15.26,14.84,0.87...|[5.24452795332028...| 0|
14.88| 14.57| 0.8811| 5.553999999999999| 3.333| 1.018| 4.956|[14.88,14.57,0.88...|[5.11393027165175...| 0|
14.29| 14.09| 0.905| 5.291|3.3369999999999997| 2.699| 4.825|[14.29,14.09,0.90...|[4.91116018695588...| 0|
13.84| 13.94| 0.8955| 5.324|3.3789999999999996| 2.259| 4.805|[13.84,13.94,0.89...|[4.75650503761158...| 0|
16.14| 14.99| 0.9034|5.6579999999999995| 3.562| 1.355| 5.175|[16.14,14.99,0.90...|[5.54696468981581...| 0|
14.38| 14.21| 0.8951| 5.386| 3.312| 2.4619999999999997| 4.956|[14.38,14.21,0.89...|[4.94209121682475...| 0|
14.69| 14.49| 0.8799| 5.563| 3.259| 3.5860000000000003| 5.218999999999999|[14.69,14.49,0.87...|[5.04863143081749...| 0|
14.11| 14.1| 0.8911| 5.42| 3.302| 2.7| 5.0|[14.11,14.1,0.891...|[4.84929812721816...| 0|
16.63| 15.46| 0.8747| 6.053| 3.465| 2.04| 5.877000000000001|[16.63,15.46,0.87...|[5.71536696354628...| 1|
16.44| 15.25| 0.888|5.8839999999999995| 3.505| 1.969|5.5329999999999995|[16.44,15.25,0.88...|[5.65006812271202...| 0|
15.26| 14.85| 0.8696|5.7139999999999995| 3.242| 4.543| 5.314|[15.26,14.85,0.86...|[5.24452795332028...| 0|
14.03| 14.16| 0.8796| 5.438| 3.201| 1.7169999999999999| 5.001|[14.03,14.16,0.87...|[4.82180387844584...| 0|
13.89| 14.02| 0.888| 5.439| 3.199| 3.986| 4.738|[13.89,14.02,0.88...|[4.77368894309428...| 0|
13.78| 14.06| 0.8759| 5.479| 3.156| 3.136| 4.872|[13.78,14.06,0.87...|[4.73588435103234...| 0|
13.74| 14.05| 0.8744| 5.482| 3.114| 2.932| 4.825|[13.74,14.05,0.87...|[4.72213722664617...| 0|
14.59| 14.28| 0.8993| 5.351| 3.333| 4.185| 4.781000000000001|[14.59,14.28,0.89...|[5.01426361985209...| 0|
13.99| 13.83| 0.9183| 5.119| 3.383| 5.234| 4.781000000000001|[13.99,13.83,0.91...|[4.80805675405968...| 0|
15.69| 14.75| 0.9058| 5.527| 3.514| 1.599| 5.046|[15.69,14.75,0.90...|[5.39230954047151...| 0|
 14.7| 14.21| 0.9153| 5.205| 3.466| 1.767| 4.649|[14.7,14.21,0.915...|[5.05206821191403...| 0|
12.72| 13.57| 0.8686| 5.226| 3.049| 4.102| 4.914|[12.72,13.57,0.86...|[4.37158555479908...| 2|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [44]:
results.select('prediction').show()

+----------+
prediction|
+----------+
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 1|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 2|
+----------+
only showing top 20 rows

In [45]:
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [46]:
dataset = spark.read.csv('/FileStore/tables/hack_data.csv',
                        inferSchema=True,
                        header=True)

In [47]:
dataset.head()

Out[ 56 ]: Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [48]:
dataset.columns

Out[ 57 ]: 
['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [49]:
feature_cols = ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [50]:
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')

In [51]:
final_data = assembler.transform(dataset)

In [52]:
final_data.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- Location: string (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)
-- features: vector (nullable = true)

In [53]:
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

In [54]:
scaler_model = scaler.fit(final_data)
cluster_final_data = scaler_model.transform(final_data)

In [55]:
cluster_final_data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted| Location|WPM_Typing_Speed| features| scaledFeatures|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+
 8.0| 391.09| 1| 2.96| 7.0| Slovenia| 72.37|[8.0,391.09,1.0,2...|[0.56785108466505...|
 20.0| 720.99| 0| 3.04| 9.0|British Virgin Is...| 69.08|[20.0,720.99,0.0,...|[1.41962771166263...|
 31.0| 356.32| 1| 3.71| 8.0| Tokelau| 70.58|[31.0,356.32,1.0,...|[2.20042295307707...|
 2.0| 228.08| 1| 2.48| 8.0| Bolivia| 70.8|[2.0,228.08,1.0,2...|[0.14196277116626...|
 20.0| 408.5| 0| 3.57| 8.0| Iraq| 71.28|[20.0,408.5,0.0,3...|[1.41962771166263...|
 1.0| 390.69| 1| 2.79| 9.0| Marshall Islands| 71.57|[1.0,390.69,1.0,2...|[0.07098138558313...|
 18.0| 342.97| 1| 5.1| 7.0| Georgia| 72.32|[18.0,342.97,1.0,...|[1.27766494049636...|
 22.0| 101.61| 1| 3.03| 7.0| Timor-Leste| 72.03|[22.0,101.61,1.0,...|[1.56159048282889...|
 15.0| 275.53| 1| 3.53| 8.0|Palestinian Terri...| 70.17|[15.0,275.53,1.0,...|[1.06472078374697...|
 12.0| 424.83| 1| 2.53| 8.0| Bangladesh| 69.99|[12.0,424.83,1.0,...|[0.85177662699757...|
 15.0| 249.09| 1| 3.39| 9.0|Northern Mariana ...| 70.77|[15.0,249.09,1.0,...|[1.06472078374697...|
 32.0| 242.48| 0| 4.24| 8.0| Zimbabwe| 67.93|[32.0,242.48,0.0,...|[2.27140433866020...|
 23.0| 514.54| 0| 3.18| 8.0| Isle of Man| 68.56|[23.0,514.54,0.0,...|[1.63257186841202...|
 9.0| 284.77| 0| 3.12| 9.0|Sao Tome and Prin...| 70.82|[9.0,284.77,0.0,3...|[0.63883247024818...|
 27.0| 779.25| 1| 2.37| 8.0| Greece| 72.73|[27.0,779.25,1.0,...|[1.91649741074455...|
 12.0| 307.31| 1| 3.22| 7.0| Solomon Islands| 67.95|[12.0,307.31,1.0,...|[0.85177662699757...|
 21.0| 355.94| 1| 2.0| 7.0| Guinea-Bissau| 72.0|[21.0,355.94,1.0,...|[1.49060909724576...|
 10.0| 372.65| 0| 3.33| 7.0| Burkina Faso| 69.19|[10.0,372.65,0.0,...|[0.70981385583131...|
 20.0| 347.23| 1| 2.33| 7.0| Mongolia| 70.41|[20.0,347.23,1.0,...|[1.41962771166263...|
 22.0| 456.57| 0| 1.52| 8.0| Nigeria| 69.35|[22.0,456.57,0.0,...|[1.56159048282889...|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+
only showing top 20 rows

In [56]:
cluster_final_data.columns

Out[ 72 ]: 
['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed',
 'features',
 'scaledFeatures']

In [57]:
kmeans2 = KMeans(featuresCol='scaledFeatures', k=2)
kmeans3 = KMeans(featuresCol='scaledFeatures', k=3)

In [58]:
model_k2 = kmeans2.fit(cluster_final_data)
model_k3 = kmeans3.fit(cluster_final_data)

In [59]:
model_k3.transform(cluster_final_data).select('prediction').show()

+----------+
prediction|
+----------+
 1|
 2|
 1|
 1|
 2|
 1|
 1|
 1|
 1|
 1|
 1|
 2|
 2|
 2|
 1|
 1|
 1|
 2|
 1|
 2|
+----------+
only showing top 20 rows

In [60]:
model_k3.transform(cluster_final_data).select('prediction').groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 83|
 2| 84|
 0| 167|
+----------+-----+

In [61]:
model_k2.transform(cluster_final_data).select('prediction').groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 0| 167|
+----------+-----+